In [148]:
import glob
from codecs import open

import pandas as pd
import numpy as np
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

%env GOOGLE_APPLICATION_CREDENTIALS=/Users/emersonsjsu/GitHub/EarlGrey/google_private_key.json
WORKING_DIRECTORY = "/Users/emersonsjsu/GitHub/EarlGrey"

env: GOOGLE_APPLICATION_CREDENTIALS=/Users/emersonsjsu/GitHub/EarlGrey/google_private_key.json


In [104]:
my_index = pd.MultiIndex(levels=[[], []],
                         labels=[[], []],
                         names=[u'character_name', u'episode_no'])
my_columns = [u'text',u'sentiment_score',u'sentiment_magnitude']
char_df = pd.DataFrame(index=my_index, columns=my_columns)
char_df

,,text,sentiment_score,sentiment_magnitude
character_name,episode_no,,,


In [ ]:
# Clear char_df
char_df = char_df.iloc[0:0]
FIRST_EPISODE_OFFSET = 101


def sanitize_name(name_line):
    name = name_line.strip()  # Voice overs count as same character
    if name.endswith('V.O.'):
        name = name[:-5]
    # Strip parentheticals from name
    if name.find('(') != -1:
        name = name[:name.find('(')]
    # Strip 'S VOICE from name
    if name.endswith('\'S VOICE'):
        name = name[:-8]
    # Strip 'S COM VOICE from name
    if name.endswith('\'S COM VOICE'):
        name = name[:-12]
    return name.strip()


for file_path in glob.glob(WORKING_DIRECTORY+'/scripts/225.txt'):  # Only read one file for debug purposes
# for file_path in glob.glob(WORKING_DIRECTORY+'/scripts/*.txt'):
    current_file = open(file_path, "r", encoding='utf-8', errors="ignore")
    episode_no = int(current_file.name[current_file.name.rfind('/') + 1:-4]) - FIRST_EPISODE_OFFSET
    lines = current_file.readlines()
    character_name = ''
    for line in lines:
        # Character dialogue has ended, reset current character
        if line.strip() == '':
            character_name = ''
        # All names are preceded by exactly 5 tabs
        elif line[:5] == '\t\t\t\t\t' and line[5:6] != '\t' and not line.strip().startswith('('):
            # We have found a name! Update current name
            character_name = sanitize_name(line)
        # If we are still looking at a character's dialogue
        elif character_name != '':
            line_text = line.strip()
            # Previous dialogue found for character this episode, append line to previous text
            if (character_name, episode_no) in char_df.index:
                char_df.loc[(character_name, episode_no), 'text'] += ' ' + line_text
            # First dialogue for character this episode, initialize string list
            else:
                char_df.loc[(character_name, episode_no), 'text'] = line_text
char_df.loc[('PICARD',124),'text']


u'Captain\'s log, stardate 45944.1. We\'re en route to Starbase Two-eighteen after a magnetic wave survey of the Parvenium Sector. We will give our report directly to Fleet Admiral Gustafson. The last time I encountered Admiral Gustafson... I ended up spending nine straight hours at the opera. The entire "Ring" cycle at one sitting... And for me... but not, apparently, for Admiral Gustafson. She went back the next day and sat through it all again. I warned her that this time -- On screen. Magnify. Mister Data? Increase power to -- Where is this... ? Computer, freeze program. (beat) End program. Picard to Enterprise -- I asked you... what is this place? Where is my uniform? My communicator? I am Captain Jean-Luc Picard of the Federation Starship Enterprise. I would like to speak with whoever is in charge here. Just tell me this -- am I a prisoner here? Are you in charge here? I want to be returned to my ship immediately. Please... if you could tell me... what is this place? Where am I? 

In [119]:
def analyze(text):
    client = language.LanguageServiceClient()
    document = types.Document(
        content=text,
        type=enums.Document.Type.PLAIN_TEXT
    )
    annotations = client.analyze_sentiment(document=document)
    return annotations.document_sentiment.score, annotations.document_sentiment.magnitude


char_df['sentiment_score'], char_df['sentiment_magnitude'] = np.vectorize(analyze)(char_df['text'])
char_df.shape

(153, 3)

In [120]:
char_df.to_pickle(WORKING_DIRECTORY+'/char_df.pickle')

In [124]:
temp = pd.read_pickle(WORKING_DIRECTORY+'/char_df.pickle')
temp

,,text,sentiment_score,sentiment_magnitude
character_name,episode_no,,,
TROI,176,"That was an incredible program. Worf, we were ...",0.1,17.400000
WORF,176,I am glad you approve. I have always found the...,0.0,15.700000
TROI,176,I should take my own advice and use it to rela...,0.5,1.600000
PICARD,176,Counselor! (urgent) What's today's date? The d...,0.0,137.000000
GEORDI,176,(calls out) Captain Picard to the Bridge! (wit...,0.0,21.100000
TASHA,176,"Captain? Are you all right, sir? Sir... ? I wa...",0.0,7.300000
COM VOICE,176,Enterprise to Shuttlecraft Galileo. You are cl...,0.0,0.100000
BEVERLY,176,I've finished the neurographic scan. I don't s...,0.0,30.799999
OGAWA,176,"The bio-spectral test results, Doctor. Doctor....",0.0,0.200000


In [164]:
char_df.loc[('PICARD',124),'text']

u'Captain\'s log, stardate 45944.1. We\'re en route to Starbase Two-eighteen after a magnetic wave survey of the Parvenium Sector. We will give our report directly to Fleet Admiral Gustafson.  4    INT. BRIDGE  PICARD, RIKER, DATA, WORF, and GEORDI at their positions; N.D.\'s as necessary. There is a light-hearted mood on the bridge.  The last time I encountered Admiral Gustafson... I ended up spending nine straight hours at the opera.  The entire "Ring" cycle at one sitting...  And for me... but not, apparently, for Admiral Gustafson. She went back the next day and sat through it all again. I warned her that this time --  STAR TREK: "The Inner Light" - 3/24/92 - TEASER       2.  4    CONTINUED:  On screen.  5    ANGLE - VIEWSCREEN (OPTICAL)  An alien object of unusual shape can be seen, though still at quite a distance. Picard moves toward the screen, between t conn and ops.  Magnify.  The object springs into greater relief. It is strange, and somehow foreboding.  Mister Data?  Increa